# Grafici Coronavirus

Come utilizzare il Notebook:

- **Importare sempre i moduli (Blocco sotto)**
- Se non si hanno i dati dei giorni passati -> eseguire **Step 0 e successivi**
- Se si hanno i dati e si vogliono quelli giornalieri -> eseguire **Step 1 e successivi**
- Se si vuole solo analizzare i grafici -> eseguire **Step 2 e successivi**

Importiamo i moduli

In [ ]:
import datetime
import pandas as pd
import os
import plotly.graph_objects as go
import wget

----------------------------------------------------------------------
----------------------------------------------------------------------

### Step 0 - Caricare dati antecedenti (SOLO PRIMA VOLTA)

La prima volta è necessario avere tutti i dati dei giorni precedenti (**non necessario dopo il primo caricamento**):

- Scaricare i dati dal sito della Protezione Civile: https://github.com/pcm-dpc/COVID-19)

In [ ]:
url = "https://www.github.com/pcm-dpc/COVID-19/trunk/dati-regioni"
os.system("svn checkout " + url)
os.remove("./dati-regioni/dpc-covid19-ita-regioni.csv")
os.remove("./dati-regioni/dpc-covid19-ita-regioni-latest.csv")

url = "https://www.github.com/pcm-dpc/COVID-19/trunk/dati-andamento-nazionale"
os.system("svn checkout " + url)
os.remove("./dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv")
os.remove("./dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale-latest.csv")

In [ ]:
regions = ["Italia","Abruzzo","Basilicata","P.A. Bolzano","Calabria","Campania","Emilia-Romagna","Friuli Venezia Giulia","Lazio","Liguria","Lombardia","Marche","Molise","Piemonte","Puglia","Sardegna","Sicilia","Toscana","P.A. Trento","Umbria","Valle d'Aosta","Veneto"]

In [ ]:
dataframe_collection = {} 
for region in regions:
    print(region)
    dataframe_collection[region] = pd.DataFrame()
    if region == "Italia":
        for filename in os.listdir("./dati-andamento-nazionale"):
            if filename.endswith(".csv"):
                f = open("./dati-andamento-nazionale/" + filename)
                temp_df = pd.read_csv(f)
                temp_df= temp_df.loc[temp_df['stato'] == "ITA"]
                dataframe_collection[region] = dataframe_collection[region].append(temp_df,"\n")
                del temp_df
    else:
        for filename in os.listdir("./dati-regioni"):
            if filename.endswith(".csv"):
                f = open("./dati-regioni/" + filename)
                temp_df = pd.read_csv(f)
                temp_df= temp_df.loc[temp_df['denominazione_regione'] == region]
                dataframe_collection[region] = dataframe_collection[region].append(temp_df,"\n")
                del temp_df
    dataframe_collection[region] = dataframe_collection[region].sort_values(by=['data'], ignore_index=True)
    
    #Create new columns, to create other plots
    dataframe_collection[region]["nuovi_deceduti"] = dataframe_collection[region]["deceduti"].diff().fillna(0).astype(int)
    dataframe_collection[region]["nuovi_guariti"] = dataframe_collection[region]["dimessi_guariti"].diff().fillna(0).astype(int)

In [ ]:
for key in dataframe_collection.keys():    
    if not os.path.exists("./dati-regioni/cumulativi/"):
        os.makedirs("./dati-regioni/cumulativi/")
    if not os.path.exists("./dati-andamento-nazionale/cumulativi/"):
        os.makedirs("./dati-andamento-nazionale/cumulativi/")

    if key == "Italia":
        dataframe_collection[key].to_csv("./dati-andamento-nazionale/cumulativi/"+ key + ".csv", index=False)
    else:
        dataframe_collection[key].to_csv("./dati-regioni/cumulativi/"+ key + ".csv", index=False)

------------------------------------------
------------------------------------------

### Step 1 - Scaricare i dati giornalieri dalla Protezione Civile (ESEGUIRE SOLO DOPO LA PUBBLICAZIONE GIORNALIERA DEI DATI - dalle h19)

Scriviamo il csv giornaliero su disco

In [ ]:
now = datetime.datetime.now()
today_is=now.strftime("%Y%m%d")
regions = ["Italia","Abruzzo","Basilicata","P.A. Bolzano","Calabria","Campania","Emilia-Romagna","Friuli Venezia Giulia","Lazio","Liguria","Lombardia","Marche","Molise","Piemonte","Puglia","Sardegna","Sicilia","Toscana","P.A. Trento","Umbria","Valle d'Aosta","Veneto"]

In [ ]:
input_filename_regioni = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-" + today_is + ".csv"
output_filename_regioni = "./dati-regioni/dpc-covid19-ita-regioni-" + today_is + ".csv"
virus_today_regioni = pd.read_csv(input_filename_regioni)
virus_today_regioni.to_csv(output_filename_regioni,index=False)

input_filename_italia = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale-" + today_is + ".csv"
output_filename_italia = "./dati-regioni/dpc-covid19-ita-andamento-nazionale-" + today_is + ".csv"
virus_today_italia = pd.read_csv(input_filename_italia)
virus_today_italia.to_csv(output_filename_italia,index=False)

Uniamo il dato giornaliero al file cumulativo, regione per regione:

In [ ]:
dataframe_collection = {}
for region in regions:
    if region == "Italia":
        today_collection = pd.DataFrame()
        dataframe_collection[region] = pd.read_csv("./dati-andamento-nazionale/cumulativi/" + region + ".csv")
        today_collection = virus_today.loc[virus_today['stato'] == "ITA"]       
    else:
        today_collection = pd.DataFrame()
        dataframe_collection[region] = pd.read_csv("./dati-regioni/cumulativi/" + region + ".csv")
        today_collection = virus_today.loc[virus_today['denominazione_regione'] == region]
    
    dataframe_collection[region]= dataframe_collection[region].append(today_collection,"\n")   
    #Create new columns, to create other plots
    dataframe_collection[region]["nuovi_deceduti"] = dataframe_collection[region]["deceduti"].diff().fillna(0).astype(int)
    dataframe_collection[region]["nuovi_guariti"] = dataframe_collection[region]["dimessi_guariti"].diff().fillna(0).astype(int)

In [ ]:
dataframe_collection["Marche"]

In [ ]:
for key in dataframe_collection.keys():
    dataframe_collection[key].to_csv("./dati-regioni/cumulativi/"+ key + ".csv", index=False)

### Step 2 - Fare i grafici (ESEGUIBILE SEMPRE)

In [ ]:
now = datetime.datetime.now()
show_time=now.strftime("%d/%m/%Y")
regions = ["Italia","Abruzzo","Basilicata","P.A. Bolzano","Calabria","Campania","Emilia-Romagna","Friuli Venezia Giulia","Lazio","Liguria","Lombardia","Marche","Molise","Piemonte","Puglia","Sardegna","Sicilia","Toscana","P.A. Trento","Umbria","Valle d'Aosta","Veneto"]

In [ ]:
dataframe_collection = {}
for region in regions:
    if region == "Italia":
        dataframe_collection[region] = pd.read_csv("./dati-andamento-nazionale/cumulativi/" + region + ".csv")
    else:
        dataframe_collection[region] = pd.read_csv("./dati-regioni/cumulativi/" + region + ".csv")
    dataframe_collection[region]["data"] = pd.to_datetime(dataframe_collection[region].data)
    dataframe_collection[region]["data"] = dataframe_collection[region]["data"].dt.strftime('%d/%m')

Creiamo qualche grafico:

In [ ]:
def incrementi_giornalieri(region):
    #Plots
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["nuovi_positivi"],
        mode='lines+markers',
        name='Nuovi Positivi',
        marker_color='orange',
        marker_size=9
    ))
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["nuovi_deceduti"],
        mode='lines+markers',
        name='Nuovi Deceduti',
        marker_color='black',
        marker_size=9
    ))

    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["nuovi_guariti"],
        mode='lines+markers',
        name='Nuovi Guariti',
        marker_color='green',
        marker_size=9
    ))

    fig.update_layout(
        title={'text': region + " - " + show_time + " - Sommario (Incrementi giornalieri)",'y':0.9,'x':0.5,'xanchor':'center','yanchor': 'top'},
        xaxis_title="Data",
        yaxis_title="Persone",
        font=dict(
        size=12),
        xaxis=dict(dtick=5)
    )

    fig.show()

In [ ]:
def situazione_attuale(region):
    #Plots
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["totale_casi"],
        mode='lines+markers',
        name='Totale Positivi',
        marker_color='red',
        marker_size=9
    ))
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["totale_positivi"],
        mode='lines+markers',
        name='Attualmente Positivi',
        marker_color='orange',
        marker_size=9
    ))
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["deceduti"],
        mode='lines+markers',
        name='Deceduti',
        marker_color='black',
        marker_size=9
    ))
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["dimessi_guariti"],
        mode='lines+markers',
        name='Guariti',
        marker_color='green',
        marker_size=9
    ))

    fig.update_layout(
        title={'text': region + " - " + show_time + " - Sommario (Incrementi giornalieri)",'y':0.9,'x':0.5,'xanchor':'center','yanchor': 'top'},
        xaxis_title="Data",
        yaxis_title="Persone",
        font=dict(
        size=12),
        xaxis=dict(dtick=6),
        yaxis=dict(hoverformat='.f')
    )

    fig.show()

In [ ]:
def situazione_positivi(region):
    #Plots
    fig = go.Figure()
    
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["totale_positivi"],
        mode='lines+markers',
        name='Totale Positivi (Somma)',
        hoverlabel = dict(namelength = -1),
        marker_color='red',
        marker_size=9
    ))
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["terapia_intensiva"],
        mode='lines+markers',
        name='Terapia Intensiva',
        hoverlabel = dict(namelength = -1),
        marker_color='purple',
        marker_size=9
    ))
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["ricoverati_con_sintomi"],
        mode='lines+markers',
        name='Ricoverati con sintomi',
        hoverlabel = dict(namelength = -1),
        marker_color='orange',
        marker_size=9
    ))
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["isolamento_domiciliare"],
        mode='lines+markers',
        name='Isolamento Domiciliare',
        hoverlabel = dict(namelength = -1),
        marker_color='blue',
        marker_size=9
    ))

    fig.update_layout(
        title={'text': region + " - " + show_time + " - Situazione Positivi",'y':0.9,'x':0.5,'xanchor':'center','yanchor': 'top'},
        xaxis_title="Data",
        yaxis_title="Persone",
        font=dict(
        size=12),
        hovermode="closest",
        xaxis=dict(dtick=6),
        yaxis=dict(hoverformat='.f')
    )

    fig.show()

In [ ]:
def carico_ospedaliero(region):
    #Plots
    fig = go.Figure()
    
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["totale_positivi"],
        mode='lines+markers',
        name='Totale Positivi (Somma)',
        hoverlabel = dict(namelength = -1),
        marker_color='red',
        marker_size=9
    ))
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["terapia_intensiva"],
        mode='lines+markers',
        name='Terapia Intensiva',
        hoverlabel = dict(namelength = -1),
        marker_color='purple',
        marker_size=9
    ))
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["ricoverati_con_sintomi"],
        mode='lines+markers',
        name='Ricoverati con sintomi',
        hoverlabel = dict(namelength = -1),
        marker_color='orange',
        marker_size=9
    ))
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["isolamento_domiciliare"],
        mode='lines+markers',
        name='Isolamento Domiciliare',
        hoverlabel = dict(namelength = -1),
        marker_color='blue',
        marker_size=9
    ))

    fig.update_layout(
        title={'text': region + " - " + show_time + " - Situazione Positivi",'y':0.9,'x':0.5,'xanchor':'center','yanchor': 'top'},
        xaxis_title="Data",
        yaxis_title="Persone",
        font=dict(
        size=12),
        hovermode="closest",
        xaxis=dict(dtick=6),
        yaxis=dict(hoverformat='.f')
    )

    fig.show()

In [ ]:
situazione_positivi("Italia")